In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, callbacks
from sklearn.model_selection import train_test_split

In [5]:
# ────────────────────────────────────────────────────────────────────────────────
# 0) Hyperparameters & Constants
# ────────────────────────────────────────────────────────────────────────────────
MAX_VOCAB_SIZE    = 20_000
MAX_SEQUENCE_LEN  = 200
EMBEDDING_DIM     = 128
LSTM_UNITS        = 64
BATCH_SIZE        = 64
EPOCHS            = 1
AUTOTUNE          = tf.data.AUTOTUNE
NUM_CLASSES       = 4
CLASS_NAMES       = ["World", "Sports", "Business", "Sci/Tech"]


In [6]:
# ────────────────────────────────────────────────────────────────────────────────
# 1) Load & preprocess AG News CSVs
# ────────────────────────────────────────────────────────────────────────────────
train_df = pd.read_csv("D:/AIML/data/ag_news_train.csv", header=None,
                       names=["label","title","description"])
test_df  = pd.read_csv("D:/AIML/data/ag_news_test.csv",  header=None,
                       names=["label","title","description"])
train_df["label"] -= 1
test_df["label"]  -= 1
train_df["text"] = train_df["title"].str.cat(train_df["description"], sep=" ")
test_df["text"]  = test_df["title"].str.cat(test_df["description"], sep=" ")


In [7]:
# ────────────────────────────────────────────────────────────────────────────────
# 2) Train/validation split
# ────────────────────────────────────────────────────────────────────────────────
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["text"].values,
    train_df["label"].values,
    test_size=0.2,
    random_state=42,
    stratify=train_df["label"].values
)
test_texts  = test_df["text"].values
test_labels = test_df["label"].values


In [8]:
# ────────────────────────────────────────────────────────────────────────────────
# 3) TextVectorization
# ────────────────────────────────────────────────────────────────────────────────
vectorizer = layers.TextVectorization(
    max_tokens=MAX_VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LEN
)
vectorizer.adapt(train_texts)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    token_ids = vectorizer(text)
    return tf.squeeze(token_ids, axis=0), label

def make_dataset(texts, labels, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((texts, labels))
    if shuffle:
        ds = ds.shuffle(len(texts), seed=42)
    ds = ds.map(vectorize_text, num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = make_dataset(train_texts, train_labels, shuffle=True)
val_ds   = make_dataset(val_texts,   val_labels)
test_ds  = make_dataset(test_texts,  test_labels)

In [9]:
# ────────────────────────────────────────────────────────────────────────────────
# 4) Build BiLSTM Functional Model
# ────────────────────────────────────────────────────────────────────────────────
inp = layers.Input(shape=(MAX_SEQUENCE_LEN,), dtype="int32", name="input_tokens")
x   = layers.Embedding(
          input_dim=MAX_VOCAB_SIZE,
          output_dim=EMBEDDING_DIM,
          input_length=MAX_SEQUENCE_LEN,
          mask_zero=True
      )(inp)
x   = layers.Bidirectional(layers.LSTM(LSTM_UNITS))(x)
x   = layers.Dropout(0.5)(x)
x   = layers.Dense(64, activation="relu")(x)
x   = layers.Dropout(0.5)(x)
out = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = Model(inputs=inp, outputs=out, name="bilstm_agnews")

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

Model: "bilstm_agnews"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tokens (InputLayer)   [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                     

In [10]:
# ────────────────────────────────────────────────────────────────────────────────
# 5) Train
# ────────────────────────────────────────────────────────────────────────────────
ckpt = callbacks.ModelCheckpoint(
    "D:/AIML/data/bilstm_txt_cls-fun.h5",
    monitor="val_accuracy",
    save_best_only=True
)
es = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[ckpt, es]
)

1500/1500 [==============================] - ETA: 0s - loss: 0.3839 - accuracy: 0.8695

d:\Softwares\anaconda\envs\homl3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1500/1500 [==============================] - 387s 254ms/step - loss: 0.3839 - accuracy: 0.8695 - val_loss: 0.2469 - val_accuracy: 0.9185


In [11]:

# ────────────────────────────────────────────────────────────────────────────────
# 6) Evaluate
# ────────────────────────────────────────────────────────────────────────────────
loss, acc = model.evaluate(test_ds)
print(f"Test accuracy: {acc:.4f}")

119/119 [==============================] - 12s 100ms/step - loss: 0.2540 - accuracy: 0.9189
Test accuracy: 0.9189


In [12]:

# ────────────────────────────────────────────────────────────────────────────────
# 7) Demo Predictions
# ────────────────────────────────────────────────────────────────────────────────
def predict(text):
    seq = vectorizer(tf.constant([text]))
    probs = model.predict(seq)[0]
    idx = int(tf.argmax(probs))
    return CLASS_NAMES[idx], float(probs[idx])

examples = [
    "NASA plans a new mission to Mars.",
    "The stock market experienced a sharp decline today."
]
for ex in examples:
    cls, conf = predict(ex)
    print(f"{cls} ({conf:.1%}): {ex}")


1/1 [==============================] - 2s 2s/step
Sci/Tech (97.9%): NASA plans a new mission to Mars.
1/1 [==============================] - 0s 30ms/step
Business (90.5%): The stock market experienced a sharp decline today.
